#PyTorch Intro - Budowa Sieci Neuronowej - Wykład

##Przygotowanie środowiska
Upewnij się, że notatnik jest uruchomiony na maszynie z GPU. Jeśli GPU nie jest dostępne zmień typ maszyny (Runtime | Change runtime type) i wybierz T4 GPU.

In [ ]:
!nvidia-smi

Biblioteka PyTorch (`torch`) jest domyślnie zainstalowana w środowisku COLAB.

In [ ]:
import torch
import numpy as np

print(f"Wersja biblioteki PyTorch: {torch.__version__}")

Sprawdzenie dostępnego urządzenia GPU.

In [ ]:
print(f"Dostępność GPU: {torch.cuda.is_available()}")
print(f"Typ GPU: {torch.cuda.get_device_name(0)}")

# Budowa sieci neuronowych w PyTorch

Sieci neuronowe składają się z warstw operujących na tensorach zawierająch przetwarzane dane.
Warstwy są podstawowymi elementami składowymi sieci neuronowych.
Przestrzeń nazw `torch.nn` zawiera implementację wielu warstw pozwalające budować sieci neuronowe o różnych architekturach (w pełni połączone, splotowe, rekurencyjne czy Transformer).
Wszystkie warstwy z biblioteki PyTorch dziedziczą z klasy `nn.Module`.

**UWAGA**: Samodzielnie zaimplementowane warstwy czy sieci złożone z wielu warstw muszą również dziedziczyć z klasy `nn.Module`.
Pozwala to tworzyć sieci o złożonej architekturze, zawierające jako składowe prostsze sieci (moduły).


##Warstwy

Lista warstw zaimplementowana w bibliotece PyTorch: [link](https://pytorch.org/docs/stable/nn.html#module-torch.nn).


Większość warstw wchodzących w skład sieci neuronowych jest parametryzowana - zawiera zestaw parametrów (wag). Parametry warstwy bądź modułu sieci neuronowej są dostępne przez metody `parameters()` lub `named_parameters()`.
Parametry (wagi) modułów sieci są przechowywane jako tensory z domyślnie włączonym śledzeniem historii obliczeń (ustawiony atrybut `requires_grad`).
Parametry modułów sieci są inicjalizowane losowo i optymalizowane w procesie uczenia sieci metodą spadku wzdłuż gradientu.

In [ ]:
import torch
import torch.nn as nn

###Warstwa liniowa
**Warstwa liniowa** `nn.Linear` stosuje przekształcenie liniowe (a właściwie afiniczne) na wejściowym tensorze $x$.
$$
y = x W^T + b
$$
gdzie $W$ jest macierzą wag a $b$ wektorem obiążenia (bias). Warsta liniowa jest elementem składowym perceptronu wielowarstwowego.
Parametry (wagi) warstwy liniowej, i wszystkich innych parametryzowanych warstw, są inicjalizowane losowo i optymalizowane w procesie uczenia.

Dokumentacja: [link](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html).


In [ ]:
def print_layer_parameters(layer: nn.Module):
    print(f"Parametry warstwy/modułu:")
    for name, param in layer.named_parameters():
        print(f"{name}: {param.shape=}  {param.requires_grad=}")

In [ ]:
linear_layer = nn.Linear(in_features=10000, out_features=8)
print(linear_layer)

x = torch.rand((4, 10000))
print(f"\nRozmiar wejściowy: {x.shape}")
y = linear_layer(x)
print(f"Rozmiar wyjściowy: {y.shape}")

print_layer_parameters(linear_layer)

###Warstwy nieliniowe

Poniżej opisanych jest kilka wybranych funkcji nieliniowych zaimplementowanych w bibliotece PyTorch.
Zauważmy, że przedstawione poniżej warstwy nieliniowe nie posiadają parametrów optymalizowanych w procesie uczenia.

Lista wszystkich warstw nieliniowych w PyToch: [link](https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity).

**Warstwa nieliniowości** `nn.ReLU` (*rectified linear unit*) stosuje do każdego elemetu tensora funkcję:
$$
\mathrm{ReLU(x)} = \max(0, x)
$$

Dokumentacja: [link](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html).

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

#Pomocnicza funkcja ilustrująca działanie warstwy nieliniowości
def test_nonlinearity(nonlienar_layer: nn.Module, label:str):
    x = torch.rand((10,))*10 - 5     # Losowy 10-elementowy tensor o wartościach z przedziału (-10, 10)
    print(f"{x=}")
    y = nonlienar_layer(x)
    print(f"{y=}")

    # Wizualizacja wyników
    x = torch.linspace(-7, 7, 100)
    y = nonlienar_layer(x)
    figure(figsize=(3, 3))
    plt.plot(x.detach().numpy(), y.detach().numpy())
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.xlabel("x")
    plt.ylabel(label)
    plt.show()

In [ ]:
nonlinear_layer = nn.ReLU()

test_nonlinearity(nonlinear_layer, "ReLU")
print_layer_parameters(nonlinear_layer)

**Warstwa nieliniowości** `nn.GELU` (*Gaussian Error Linear Unit*) stosuje do każdego elemetu tensora funkcję:
$$
\mathrm{GELU(x)} = x \cdot \Phi \left( x \right) \, ,
$$
gdzie $\Phi$ jest dystrybuantą rozkładu normalnego.
Dokumentacja: [link](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html#torch.nn.GELU).

In [ ]:
nonlinear_layer = nn.GELU()
test_nonlinearity(nonlinear_layer, "GELU")
print_layer_parameters(nonlinear_layer)

**Warstwa nieliniowości** `nn.SiLU` (*Sigmoid Linear Unit*, zwana również *swish*) stosuje do każdego elemetu tensora funkcję:
$$
\mathrm{SiLU(x)} = x \cdot \sigma \left( x \right) \, ,
$$
gdzie $\sigma$ jest funkcją logistyczną (sigmoidalną) zdefiniowaną:
$$
\sigma(x) = \frac{1}{1+e^{-x}}
$$
Dokumentacja: [link](https://pytorch.org/docs/stable/generated/torch.nn.SiLU.html).

In [ ]:
nonlinearz_layer = nn.SiLU()
test_nonlinearity(nonlinearz_layer, "SiLU")
print_layer_parameters(nonlinear_layer)

**Warstwa nieliniowości** `nn.Sigmoid` stosuje do każdego elemetu tensora funkcję logistyczną (sigmoidalną):
$$
\mathrm{Sigmoid(x)} = \sigma(x) =  \frac{1}{1+e^{-x}}
$$
Dokumentacja: [link](https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html#torch.nn.Sigmoid).

In [ ]:
nonlinearz_layer = nn.Sigmoid()
test_nonlinearity(nonlinearz_layer, "Sigmoid")
print_layer_parameters(nonlinear_layer)

**Warstwa nieliniowości** `nn.Tanh`  stosuje do każdego elemetu tensora funkcję tangensa hiperbolicznego:
$$
\mathrm{tanh(x)} = \frac{e^x - e^{-x}}{e^x + e^{-x}}
$$
Dokumentacja: [link](https://pytorch.org/docs/stable/generated/torch.nn.Tanh.html#torch.nn.Tanh).

In [ ]:
nonlinearz_layer = nn.Tanh()
test_nonlinearity(nonlinearz_layer, "Tanh")
print_layer_parameters(nonlinear_layer)

###Warstwy normalizacji

**Normalizacja wsadu** stosowana jest do normalizacji wartości przetwarzanych wektorów cech poprzez odjęcie średniej i podzielenie przez odchylenie standardowe. Pozwala to ustabilizować i przyśpieszyć proces uczenia i może poprawić generalizację modelu. Często stosowana jest w sieciach splotowych. Do dobrego działania wymaga wykorzystania dużych wsadów podczas trenowania modelu.

Warstwa normalizacji wsadu zdefiniowana jest wzorem:

$$
y = \frac{x - \mathbb{E}[x]}{\sqrt{\mathrm{Var}[x] + \epsilon}} \cdot \gamma + \beta
$$
$\gamma$ i $\beta$ są wyuczonymi parametrami rozmiaru $C$, gdzie $C$ jest liczbą cech albo kanałów wejścia.
* **W fazie treningu** - wartość oczekiwana i wariancja obliczane są po elementach wsadu. Dodatkowo obliczana i zapamiętywana jest średnia krocząca wartości oczekiwanej i wariancji dla wszystkich przetworzonych wsadów.
* **W fazie inferencji** - wykorzystywana jest wyznaczona w fazie treningu średnia krocząca wartości oczekiwanej i wariancji.

Warstwa `nn.BatchNorm1d` akceptuje tensor rozmiaru $(N, C)$ lub $(N, C, L)$ i zwraca tensor tego samego rozmiaru. Dokumentacja: [link](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm1d.html#torch.nn.BatchNorm1d).

Do normalizacji danych obrazowych stosowana jest warstwa `nn.BatchNorm2d` przetwarzająca mapy cech wizyjnych rozmiaru $(N,C,H,W)$.

In [ ]:
def print_batchnorm_params(layer: nn.Module):
    print(f"Parametry warstwy:")
    print(f"Średnia krocząca wartości oczekiwanej: {layer.running_mean}")
    print(f"Średnia krocząca wariancji: {layer.running_var}")

In [ ]:
# Defnicja warstwy BatchNorm1d
num_features = 5                # Liczba cech (kanałów)
batch_norm = nn.BatchNorm1d(num_features)
# Domyślnie warstwa tworzona jest w trybie treningowym (atrybut training ustawiony na True)
print(f"Tryb treningowy: {batch_norm.training=}")
print_batchnorm_params(batch_norm)

# Utwórz losowy tensor
batch_size = 3
input_tensor = torch.randn(batch_size, num_features)

print("\nWejściowy tensor:")
print(input_tensor)

# Zastosuj BatchNorm1d
output_tensor = batch_norm(input_tensor)

print("\nPo zastosowaniu normalizacji wsadu:")
print(output_tensor)

print("\nŚrednie po wymiarze 0 (rozmiar wsadu):")
print(output_tensor.mean(dim=0))
print(f"Odchylenie standardowe po wymiarze 0 (rozmiar wsadu): {output_tensor.std(dim=0)}")
print()

print("\nŚrednie po wymiarze 1 (elementy wektorów cech/kanały):")
print(output_tensor.mean(dim=1))
print(f"Odchylenie standardowe po wymiarze 1 (elementy wektorów cech/kanały): {output_tensor.std(dim=1)}")
print()


print_batchnorm_params(batch_norm)

**Normalizacja warstwy** `nn.LayerNorm` podobnie jak `nn.BatchNorm1d` stosowana jest do normalizacji wartości przetwarzanych wektorów cech poprzez odjęcie średniej i podzielenie przez odchylenie standardowe.
W odróżnieniu od warstwy `nn.BatchNorm1d` normalizacja przebiega po wymiarze cech (kanałów). Wartość oczekiwana i średnia wyznaczana jest dla każdego elementu wsadu z osobna.
Normalizacja warstwy często stosowana jest w sieciach rekurencyjnych lub opartych o architekturę Transformer.

Zdefiniowana jest wzorem:

$$
y = \frac{x - \mathbb{E}[x]}{\sqrt{\mathrm{Var}[x] + \epsilon}} \cdot \gamma + \beta \, ,
$$
gdzie $\gamma$ i $\beta$ są wyuczonymi parametrami.

Dokumentacja: [link](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html#torch.nn.LayerNorm)

In [ ]:
# Defnicja warstwy LayerNorm

num_features = 5                # Liczba cech (kanałów)
layer_norm = nn.LayerNorm(num_features)

# Utwórz losowy tensor
batch_size = 3
input_tensor = torch.randn(batch_size, num_features)

print("\nWejściowy tensor:")
print(input_tensor)

# Zastosuj BatchNorm1d
output_tensor = layer_norm(input_tensor)

print("\nPo zastosowaniu normalizacji warstwy:")
print(output_tensor)

print("\nŚrednie po wymiarze 0 (rozmiar wsadu):")
print(output_tensor.mean(dim=0))
print(f"Odchylenie standardowe po wymiarze 0 (rozmiar wsadu): {output_tensor.std(dim=0)}")
print()

print("\nŚrednie po wymiarze 1 (elementy wektorów cech/kanały):")
print(output_tensor.mean(dim=1))
print(f"Odchylenie standardowe po wymiarze 1 (elementy wektorów cech/kanały): {output_tensor.std(dim=1)}")
print()


**Warstwa odrzutu** `nn.Dropout` jest metodą regularyzacji wykorzystywaną w celu ograniczenia przeuczenia i polepszenia generalizacji sieci neuronowej.

*   **W fazie treningu** zerowane są losowo wybrane elementy przetwarzanego tensora. Elementy przetwarzanego tensora wygaszane są niezależnie, każdy z niewielkim prawdopodobieństwem $p$. Dodatkowo wyjście jest skalowane ze współczynnikiem $\frac{1}{1-p}$.
*   **W fazie inferencji** warstwa jest przekształceniem identycznościowym.

Dokumentacja: [link](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html).

### Inne warstwy

Warstwa `nn.Flatten` pozwala zlinearyzować (spłaszczyć) wejściowy tensor do mniejszej liczby wymiarów.
Na przykład może zostać wykorzystana aby przekształcić obraz danych jako tensor $(n=4, c=3, h=64, w=64)$, gdzie $n$ jest rozmiarem wsadu, $c$ liczbą kanałów a $h, w$ to rodzielczość do rozmiarów $(n=4, c \cdot h \cdot w = 12\, 228)$.

Dokumentacja: [link](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html).

In [ ]:
image_height = 64
image_width = 64
image_tensor = torch.rand((4, 3, image_height, image_width))
print(f"Rozmiar wejściowy: {image_tensor.shape}")

flatten = nn.Flatten()
flat_image = flatten(image_tensor)
print(f"Rozmiar wejściowy: {flat_image.shape}")


**Warstwa sekwencyjna** `nn.Sequential` pozwala zgrupować wiele połączonych sekwencyjnie warstw i traktować je jak jedną warstwę. Dane są przetwarzane przez poszczególne moduły w kolejności zdefiniowanej przy tworzeniu warstwy sekwencyjnej.

Przykład - **perceptron wielowarstwowy** (MLP - *multi layer perceptron*) składa się z kilku warstw liniowych połączonych warstwami nieliniowymi (np. `nn.ReLU`). Perceptron wielowarstwowy możemy zaimplementować jako warstwę sekwencyjną złożoną z trzech warstw liniowych oddzielonych nieliniowością `nn.ReLU`.

Zastosowanie warstw nieliniowych jest konieczne aby perceptron definiował nieliniową funkcję wejściowego tensora. Bezpośrednie połączenie kilku warstw liniowych, bez nieliniowości, jest przekształceniem linowym (może zostać zastąpione równoważną pojedynczą warstwą `nn.Linear`).

In [ ]:
my_mlp = nn.Sequential(
    nn.Linear(10000, 256),
    nn.ReLU(),
    nn.Linear(256, 16),
    nn.ReLU(),
    nn.Linear(16, 4)
    )

print(my_mlp)
print()

input_tensor = torch.rand(2, 10000)
print(f"{input_tensor.shape=}")
logits = my_mlp(input_tensor)
print(f"{logits.shape=}")

**Warstwa softmax** `nn.Softmax` stosuje funkcję Softmax wzdłuż podanego wymiaru wejściowego tensora. Stosowana do przekształcenia nieznormalizowanych wartości rzeczywistych zwracach przez sieć (zwanych logitami) w rozkład prawdopodobieństwa klas. Wynikowe wartości są z zakresu $[0, 1]$ i sumują się do $1$.

$$\mathrm{Softmax} \left( x_i \right)
=
\frac{\exp(x_i)}{\sum_j \exp(x_j)}
 $$

**UWAGA:** Warstwa Softmax nie powinna być używana wewnątrz modułu klasyfikatora do wyznaczenia rozkładu prawdopodobieństwa klas.
Klasyfikator oparty o sieć neuronową powinien zwracać nieznormalizowane wartości rzeczywiste (logity). Aby zwiększyć stabilność numeryczną funkcja straty `nn.CrossEntropyLoss` wykorzystywana w treningu klasyfikatorów oczekuje na wejściu nieznormalizowanych wartości (logitów) a NIE rozkładu prawdopodobieństwa.


Dokumentacja: [link](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html#softmax).

In [ ]:
print(f"{logits=}")
print(f"Suma wartości logitów w wierszach: {logits.sum(dim=1)} \n")

softmax = nn.Softmax(dim=1)

probabilities = softmax(logits)
print(f"{probabilities}")
print(f"Suma wartości w wierszach po zastosowaniu softmax: {probabilities.sum(dim=1)=}\n")

print_layer_parameters(softmax)

#Przykład: Perceptron wielowarstwowy

W tym przykładzie zastosujemy **perceptron wielowarstwowy** do rozwiązania problemu klasyfikacji zbioru zawierającego syntetycznie wygenerowane punkty danych $x \in \mathbb{R}^{10}$ należące do dwóch klas.
We wcześniejszym notatniku do klasyfikacje tych danych zastosowaliśmy metodę regresji logistycznej, zaimplementowaną z wykorzystaniem pojedycznej warstwy liniowej. Ponieważ punkty danych nie były liniowo separowalne skuteczność regresji logitycznej była ograniczona.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Urządzenie: {}".format(device))

##Generowanie syntetycznych danych
Wygenerujmy zbiór zawierający syntetycznie wygeneorwane punkty danych $x \in \mathbb{R}^{10}$ należące do dwóch klas korzystając z funkcji `make_classification` z biblioteki scikit-learn ([link](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html)).

In [ ]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

# Wygeneruj syntetyczny zbiór danych
X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, n_informative=5, random_state=39)

# Podziel na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(f"Liczba elementów w zbiorze danych: {X.shape[0]}")
print(f"Liczba cech: {X.shape[1]}")
print(f"Liczba klas: {len(np.unique(y))}")

In [ ]:
print(X[:2])
print(y[:2])

Wizualizacja dwóch wybranych cech. Możemy zauważyć, że dla dwóch losowo wybranych cech elementy z różnych klas nie są liniowo separowalne.

In [ ]:
import random
from matplotlib import pyplot as plt

# Wybierz dwie losowe cechy
f1 = random.randint(0, X.shape[1]-1)
f2 = random.randint(0, X.shape[1]-1)

plt.scatter(X[y==0, f1], X[y==0, f2], color='blue', label='Klasa 0', alpha=0.7)
plt.scatter(X[y==1, f1], X[y==1, f2], color='red', label='Klasa 1', alpha=0.7)
plt.grid(True, linestyle='--', alpha=0.6)
plt.xlabel(f"Wartość cechy {f1}")
plt.ylabel(f"Wartość cechy {f2}")
plt.legend()
plt.show()


##Implementacja sieci neuronowej

W PyTorch wszystkie klasy implementujące moduły sieci neuronowej dziedziczą z klasy `nn.Module`. Jeśli piszemy własną klasę definiującą moduł sieci neuronowej **musi ona również dziedziczyć z klasy `nn.Module`**.
Utworzymy klasę `MyMLP` implementujący wielowarwarstwowy perceptron złożony z dwóch warstw liniowych `nn.Linear` i nieliniowości `ReLU`.
Aby zmniejszyć ryzyko przeuczenia i poprawić generalizację sieci przed ostatnią warstwą liniową zastosujemy warstwę odrzutu `nn.Dropout`.

*   W metodzie `__init__` inicjalizujemy warstwy i moduły wchodzących w skład sieci. Nic nie stoi na przeszkodzie aby częścią składową sieci była inna, wcześniej utworzona sieć.
*   W metodzie `forward` definiujemy logikę przetwarzania danych przez sieć. Jakie operacje i w jakiej kolejności są wykonywane na wejściowych danych. Najczęściej metoda `forward` otrzymuje jako argument pojedyczny tensor (ale teoretycznie może otrzymać zestaw kilku tensorów) i zwraca wynikowy tensor (lub tensory).

**UWAGA:** Kolejność definicji warstw w metodzie `__init__` nie określa kolejności w jakiej dane będą przetwarzane przez kolejne warstwy. Logikę przetwarzania dediniujemy w metodzie `forward`.

In [ ]:
class MyMLP(nn.Module):
    def __init__(self, n_features: int, n_classes: int):
        super().__init__()
        self.fc1 = nn.Linear(n_features, 5)
        self.fc2 = nn.Linear(5, n_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

Utworzymy instancję klasy `MyMLP`. Jako argumenty podamy liczbę cech w zbiorze danych (`n_features`=10) oraz liczbę klas (`n_classes`=2)
Dla porównaniu utworzymy również prostą sieć złożoną z jednej warstwy liniowej.
Zauważmy, że zarówno dostępna w bibliotece PyTorch warstwa `nn.Linear` jak i zaimplementowana przez nas klasa `MyMLP` są pochodnymi klasy `nn.Module`.

In [ ]:
import torch.nn as nn

n_features = X.shape[1]     # Liczna cech

mlp_net = MyMLP(n_features, 2)
linear_net = nn.Linear(n_features, 2)

print(f"{isinstance(linear_net, nn.Module)=}")
print(f"{isinstance(mlp_net, nn.Module)=}")

In [ ]:
print(mlp_net)

In [ ]:
print(linear_net)

Sprawdźmy działanie obu sieci. Każda z nich po przetworzeniu wsadu wektorów cech $X \in \mathbb{R}^{N \times 10}$ zwraca tensor liczb rzeczywistych rozmiaru $(N, 2)$.
Zauważmy, że tensor `z` zawierający wynikowe wartości ma włączone śledzenie obliczeń (`requires_grad=True`).

**UWAGA:** Aby przetworzyć dane przez moduł sieci NIE należy wywoływać bezpośrednio metody `forward`. Podajemy dane bezpośrednio do modelu, na przykład `my_model(X)`. Pozwala to oprócz przejścia w przód (*forward*) wykonać dodatkowe operacje niezbędne do prawidłowego działania sieci.

In [ ]:
# Zamień dane (jako tablice ndarray) na tensory
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.int64)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.int64)

z = linear_net(X_train_tensor)
print("linear_net:")
print(f"Rozmiar wejścia: {X_train_tensor.shape}")
print(f"Rozmiar wyjścia: {z.shape}")
print(f"Wyjście (pierwsze 5 elementów): {z[:5]}")
print(f"{z.requires_grad=}\n")

z = mlp_net(X_train_tensor)
print("mlp_net:")
print(f"Rozmiar wejścia: {X_train_tensor.shape}")
print(f"Rozmiar wyjścia: {z.shape}")
print(f"Wyjście (pierwsze 5 elementów): {z[:5]}")
print(f"{z.requires_grad=}")

W naszym przypadku wyjścia z sieci możemy je traktować jako nieznormalizowane rozkłady prawdopodobieństwa każdej klasy.
W literaturze poświęconej głębokiemu uczeniu nieznormalizowane wyjścia z sieci przyjmujące wartości rzeczywiste z zakresu $\left( -\infty, \infty \right)$ zwane są **logitami**.
Aby zamienić wyjścia z sieci (logity) na rozkład prawdopodobieństwa klas należy zastosować funkcję softmax.

**WAŻNE**: Nie należy normalizować wartości zwracanych przez sieć, na przykład stosując funkcję softmax lub sigmoid  wewnątrz modułu sieci. Ze względu na stabilność numeryczną funkcje straty zaimplementowane w PyTorch, takie jak entropia krzyżowa, domyślnie operują na nieznormalizowanych wartościach (logitach).

In [ ]:
print(f"Surowe wyjścia z sieci (logity)")
print(f"{z=}")
print(f"\nWyjścia z sieci po normalizacji (softmax)")
print(f"{torch.nn.functional.softmax(z)=}")


In [ ]:
def calc_accuracy(logits: torch.Tensor, y_true: torch.Tensor) -> float:
    # Oblicz dokładność klasyfikacji na podstawie zwróconych przez sieć nieznormalizowanych wartości (logitów)
    y_pred_labels = torch.argmax(logits, dim=1).float()
    accuracy = (y_pred_labels == y_true).sum().item() / y_true.size(0)
    return accuracy

Funkcja `train_network` implementuje prostą pętlę treningową modelu sieci neuronowej.

*   Trening modelu jest podzielony na **epoki**. W czasie jednej epoki następuje jednokrotne przejście przez zbiór danych.
*   Zwykle w każdej epoce iteracyjnie przetwarzane są kolejne **wsady** ze zbioru danych. W naszym przypadku, ponieważ zbiór danych jest względnie mały, wsad zawiera wszystkie dane.
*   Przetworzenie jednego wsadu (w naszym przypadku wszystkich danych) składa się z następujących kroków:
    * Przetworzenie elementów wsadu przez sieć (przejście w przód): `logits = model(X_train_tensor)`
    *   Wyznaczenie wartość funkcji straty `loss = criterion(logits, y_train_tensor)`
    *   Wyznaczanie gradientu funkcji straty względem parametrów sieci (przejście w tył): `loss.backward()`
    *   Krok optymalizacji parametrów sieci: `optimizer.step()`

Jako funkcję straty wykorzystamy `CrossEntropyLoss`, łączącą w jednej klasie funkcję Softmax i funkcję straty binarnej entropii krzyżowej (patrz: [link](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html#torch.nn.CrossEntropyLoss)).

**WAŻNE - specyfika PyTorcha:**

*   Domyślnie gradienty zapamiętywane w tensorach podlegających optymalizacji (wagach sieci) akumulują się po każdym wywołaniu przejścia w tył (metody `backward`). Dlatego konieczne jest wyzerowanie gradientów poleceniem `optimizer.zero_grad()`. Więcej informacji: [link](https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch).
*   Moduł sieci neuronowej może być w trybie treningowym (`train`) lub ewaluacyjnym (`eval`). W fazie trenowania model musi zostać przełączony w tryb treningowy poleceniem `model.train()`. Przy wykorzystaniu modelu do wnioskowania (np. przy wyznaczaniu dokładności klasyfikacji na zbiorze testowym) należy przełączyć model w tryb ewaluacji poleceniem `model.eval()`.
Różnice między trybem `train` i `eval`: [link](https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch).
*   Podczas wyznaczania dokładności klasyfikacji na zbiorze testowym nie musimy budować grafu obliczeń - nie potrzebujemy wyznaczać gradientu funkcji straty. Wyłączymy budowania grafu obliczeń przy użyciu menadżera kontekstu
`with torch.no_grad():` co zmniejszy wykorzystanie zasobów obliczeniowych.


In [ ]:
def train_network(model: nn.Module, criterion: nn.Module, optimizer: torch.optim.Optimizer,
                  eta: float = 0.2, n_epochs: int = 200):
    model.train()
    for k in range(n_epochs):
        optimizer.zero_grad()

        logits = model(X_train_tensor)
        # logits ma rozmiar (N, 1)
        logits = logits.squeeze(1)
        # logits ma rozmiar (N,)

        # Wyznacz wartość funkcji straty
        loss = criterion(logits, y_train_tensor)

        # Przejście w tył- wyznaczenie gradientu funkcji straty względem parametrów (wag) modelu
        loss.backward()

        # Krok optymalizacji metodą spadku wzdłuż gradientu
        optimizer.step()

        train_accuracy = calc_accuracy(logits, y_train_tensor)

        # Oblicz dokładność klasyfikacji na zbiorze testowym
        with torch.no_grad():
            model.eval()
            logits = model(X_test_tensor)
            model.train()
            test_accuracy = calc_accuracy(logits.squeeze(1), y_test_tensor)

        if k % 20 == 0:
            print(f"Epoch: {k}   Wartość funkcji straty: {loss.item():.5f}   Dokładność (train): {train_accuracy:.4f}   Dokładność (test): {test_accuracy:.4f}")

Trening i ewaluacja klasyfikatora `linear_net` opartego o pojedynczą warstwę liniową (regresja logistyczna).

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(linear_net.parameters(), lr=0.1)
train_network(linear_net, criterion, optimizer)

Trening i ewaluacja klasyfikatora opartego o perceptron wielowarstwowy `mlp_net`.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp_net.parameters(), lr=0.1)
train_network(mlp_net, criterion, optimizer)

Dla porównania sprawdzimy skuteczność klasyfikatora regresji logistycznej z biblioteki scikit-learn.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
y_test_pred = model.predict(X_test)
test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Dokładność (train): {train_accuracy:.4f}   Dokładność (test): {test_accuracy:.4f}")